In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU detectada:", gpus)
    except RuntimeError as e:
        print("Error en la configuración de GPU:", e)
else:
    print("No se detectó GPU. TensorFlow está usando CPU.")

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

In [ ]:
import os 
from pathlib import Path

path = Path('./DATASET2')

## Train
trainimgs = [str(path/'Train_images'/i) for i in os.listdir(path/'Train_images')]
trainmasks = [str(path/'Train_masks'/i) for i in os.listdir(path/'Train_masks')]
train_data = list(zip(trainimgs, trainmasks))

## Val
valimgs = [str(path/'Val_images'/i) for i in os.listdir(path/'Val_images')]
valmasks = [str(path/'Val_masks'/i) for i in os.listdir(path/'Val_masks')]
val_data = list(zip(valimgs, valmasks))

## Test
testimgs = [str(path/'Test_images'/i) for i in os.listdir(path/'Test_images')]
testmasks = [str(path/'Test_masks'/i) for i in os.listdir(path/'Test_masks')]
test_data = list(zip(testimgs, testmasks))

print(' - Train Set')
print('Número de imágenes en train:', len(trainimgs))
print('Número de máscaras en train:', len(trainmasks))
print()
print(' - Val Set')
print('Número de imágenes en val:', len(valimgs))
print('Número de máscaras en val:', len(valmasks))
print()
print(' - Test Set')
print('Número de imágenes en test:', len(testimgs))
print('Número de mascaras en test:', len(testmasks))
print()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Fijar la semilla
tf.random.set_seed(123)

# Tamaños entrada y número de categorías
img_size = (256,256,3) # El tamaño de cada patch
nclases = 1

# ===============
# Entrada
entrada = tf.keras.Input(shape=img_size)

# ===============
# Codificador

# conv1
conv1 = layers.Conv2D(16,(3,3),activation='relu',padding='same')(entrada)
conv1 = layers.Conv2D(16,(3,3),activation='relu',padding='same')(conv1)
pool1 = layers.MaxPool2D((2,2))(conv1)

# conv2
conv2 = layers.Conv2D(32,(3,3),activation='relu',padding='same')(pool1)
conv2 = layers.Conv2D(32,(3,3),activation='relu',padding='same')(conv2)
pool2 = layers.MaxPool2D((2,2))(conv2)

# conv3
conv3 = layers.Conv2D(64,(3,3),activation='relu',padding='same')(pool2)
conv3 = layers.Conv2D(64,(3,3),activation='relu',padding='same')(conv3)
pool3 = layers.MaxPool2D((2,2))(conv3)

# conv4
conv4 = layers.Conv2D(128,(3,3),activation='relu',padding='same')(pool3)
conv4 = layers.Conv2D(128,(3,3),activation='relu',padding='same')(conv4)
pool4 = layers.MaxPool2D((2,2))(conv4)

# conv5
conv5 = layers.Conv2D(256,(3,3),activation='relu',padding='same')(pool4)
conv5 = layers.Conv2D(256,(3,3),activation='relu',padding='same')(conv5)

# ===============
# Decodificador

# dec1
dec1 = layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(conv5)
dec1 = layers.concatenate([dec1,conv4])
dec1 = layers.Conv2DTranspose(128, (3,3), activation='relu', padding='same')(dec1)
dec1 = layers.Conv2DTranspose(128, (3,3), activation='relu', padding='same')(dec1)

# dec2
dec2 = layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(dec1)
dec2 = layers.concatenate([dec2,conv3])
dec2 = layers.Conv2DTranspose(64, (3,3), activation='relu', padding='same')(dec2)
dec2 = layers.Conv2DTranspose(64, (3,3), activation='relu', padding='same')(dec2)

# dec3
dec3 = layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(dec2)
dec3 = layers.concatenate([dec3,conv2])
dec3 = layers.Conv2DTranspose(32, (3,3), activation='relu', padding='same')(dec3)
dec3 = layers.Conv2DTranspose(32, (3,3), activation='relu', padding='same')(dec3)

# dec4
dec4 = layers.Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(dec3)
dec4 = layers.concatenate([dec4,conv1])
dec4 = layers.Conv2DTranspose(16, (3,3), activation='relu', padding='same')(dec4)
dec4 = layers.Conv2DTranspose(16, (3,3), activation='relu', padding='same')(dec4)

# ===============
# Salida
salida = layers.Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid')(dec4)
# ===============
# Interconectar todo en un modelo
unet = tf.keras.models.Model(inputs=entrada, outputs=salida)
unet.summary()

In [ ]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

In [ ]:
unet.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', dice_coef]
)

In [ ]:
# Si las rutas están en formato Path, conviértelas a cadenas.
trainimgs = [str(img) for img in trainimgs]
trainmasks = [str(msk) for msk in trainmasks]
valimgs = [str(img) for img in valimgs]
valmasks = [str(msk) for msk in valmasks]
testimgs = [str(img) for img in testimgs]
testmasks = [str(msk) for msk in testmasks]

trainimgs, trainmasks = zip(*train_data)
valimgs, valmasks = zip(*val_data)
testimgs, testmasks = zip(*test_data)

trainimgs = list(trainimgs)
trainmasks = list(trainmasks)
valimgs = list(valimgs)
valmasks = list(valmasks)
testimgs = list(testimgs)
testmasks = list(testmasks)

In [ ]:
# Ahora crea los datasets como lo mencionamos antes
ds_train = tf.data.Dataset.from_tensor_slices((trainimgs, trainmasks))
ds_train = ds_train.shuffle(buffer_size=1000).batch(32)

ds_val = tf.data.Dataset.from_tensor_slices((valimgs, valmasks))
ds_val = ds_val.batch(32)

ds_test = tf.data.Dataset.from_tensor_slices((testimgs, testmasks))
ds_test = ds_test.batch(32)

import tensorflow as tf
from pathlib import Path

# Fijar directorios para las imágenes y las máscaras
DIR_IMGS = Path('./DATASET/Train_images/')
DIR_MSKS = Path('./DATASET/Train_mask/')


In [ ]:
# La función para lectura y pre-procesamiento
def preprocesar_imagen(imgpath, mskpath):
    # Crear rutas completas a partir de las rutas relativas
    # Leer las imágenes y las máscaras como archivos
    img = tf.io.read_file(imgpath)  # Ahora imgpath es una cadena
    msk = tf.io.read_file(mskpath)  # Ahora mskpath es una cadena

    img = tf.image.decode_jpeg(img, channels=3)  # uint8
    msk = tf.image.decode_png(msk, channels=1)   # uint8

    img = tf.image.convert_image_dtype(img, dtype=tf.float32) 

    msk = tf.image.convert_image_dtype(msk, dtype=tf.float32)

    return img, msk

In [ ]:
# Aplicar preprocesamiento y crear los lotes
ds_train = tf.data.Dataset.from_tensor_slices((trainimgs, trainmasks))
ds_train = ds_train.map(lambda img, msk: preprocesar_imagen(img, msk))  # Mapeamos el preprocesamiento
ds_train = ds_train.shuffle(buffer_size=1000).batch(32)

ds_val = tf.data.Dataset.from_tensor_slices((valimgs, valmasks))
ds_val = ds_val.map(lambda img, msk: preprocesar_imagen(img, msk))  # Mapeamos el preprocesamiento
ds_val = ds_val.batch(32)

ds_test = tf.data.Dataset.from_tensor_slices((testimgs, testmasks))
ds_test = ds_test.map(lambda img, msk: preprocesar_imagen(img, msk))  # Mapeamos el preprocesamiento
ds_test = ds_test.batch(32)

In [ ]:
# Y entrenar el modelo: ¡en este punto SÍ se leen las imágenes!
print ("Comenzando entrenamiento")

history = unet.fit(ds_train, validation_data = ds_val, epochs=200, verbose=2)

print ("Entrenamiento finalizado con éxito")

## Guardado de modelo

In [ ]:
unet.save(('Modelo_Final.keras'))

## Prueba del modelo

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel("# Epoca")
plt.ylabel("Magnitud de pérdida")
plt.plot(history.history["loss"])

In [ ]:
import matplotlib.pyplot as plt

# Obtener una imagen del dataset de validación
for img, msk in ds_val.take(1):
    pred = unet.predict(img)

    for i in range(3):  # Mostrar 3 ejemplos
        plt.figure(figsize=(12,4))
        
        plt.subplot(1,3,1)
        plt.title("Imagen")
        plt.imshow(img[i])
        
        plt.subplot(1,3,2)
        plt.title("Máscara real")
        plt.imshow(msk[i,...,0], cmap='gray')
        
        plt.subplot(1,3,3)
        plt.title("Predicción")
        plt.imshow(pred[i,...,0], cmap='gray')
        
        plt.show()
